In [ ]:
!pip install pydub
!pip install librosa
!pip install numba==0.49.0
!pip install llvmlite==0.32.1

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import librosa
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg
from pydub import AudioSegment
import shutil
from keras.preprocessing.image import ImageDataGenerator
import random

# COLAB = True
COLAB = False

In [ ]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    try:
        os.makedirs('/content/gdrive/MyDrive/Data/spectrograms')
    except FileExistsError:
        pass
else:
    try:
        os.makedirs('./spectrograms')
    except FileExistsError:
        pass


In [ ]:
genres = 'blues classical country disco pop hiphop metal reggae rock'
genres = genres.split()

In [ ]:
if COLAB:
    for g in genres:
        path1 = os.path.join('/content/gdrive/MyDrive/Data/audio_samples', f'{g}')
        try:
            os.makedirs(path1)
        except FileExistsError:
            pass
        path = os.path.join(
            '/content/gdrive/MyDrive/Data/spectrograms', f'{g}')
        try:
            os.makedirs(path)
        except FileExistsError:
            pass
else:
    for g in genres:
        path1 = os.path.join('./audio_samples', f'{g}')
        try:
            os.makedirs(path1)
        except FileExistsError:
            pass
        path = os.path.join('./spectrograms', f'{g}')
        try:
            os.makedirs(path)
        except FileExistsError:
            pass


In [ ]:
from pydub import AudioSegment
newSong = 'newSong'
i = 0
for g in genres:
    j = 0
    print(f"{g}")
    if COLAB:
        directory = '/content/gdrive/MyDrive/Data/genres_original'
    else:
        directory = './genres_original'
    for filename in os.listdir(os.path.join(directory, f"{g}")):
        song = os.path.join(f'{directory}/{g}', f'{filename}')
        j += 1
        for w in range(0, 10):
            i += 1
            t1 = 3 * w * 1000
            t2 = 3 * (w + 1) * 1000
            newAudio = AudioSegment.from_wav(song)
            new = newAudio[t1:t2]
            if COLAB:
                new.export(
                    f'/content/gdrive/MyDrive/Data/audio_samples/{g}/{g}{j}{w}.wav', format="wav")
            else:
                new.export(
                    f'./audio_samples/{g}/{g}{j}{w}.wav', format="wav")

In [ ]:
if COLAB:
    !zip -r /content/audio_samples.zip /content/audio_samples

In [ ]:
if COLAB:
    directory = '/content/audio_samples'
else:
    directory = './audio_samples'
for g in genres:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join(directory, f"{g}")):
        song = os.path.join(f'{directory}/{g}', f'{filename}')
        j = j+1
        y,sr = librosa.load(song, duration=3)
        mels = librosa.feature.melspectrogram(y=y, sr=sr)
        fig = plt.Figure()
        canvas = FigureCanvasAgg(fig)
        p = plt.imshow(librosa.power_to_db(mels, ref=np.max))
        if COLAB:
            plt.savefig(
                f'/content/gdrive/MyDrive/spectrograms/{g}/{g}{j}.png')
        else:
            plt.savefig(f'./spectrograms/{g}/{g}{j}.png')

In [ ]:
# Split data into testing and training
directory = './spectrograms/train/'
if COLAB:
    directory = "/content/gdrive/MyDrive/Data/spectrograms/train/"
for g in genres:
    filenames = os.listdir(os.path.join(directory, f"{g}"))
    random.shuffle(filenames)
    test_files = filenames[0:100]
    for f in test_files:
        shutil.move(f"{directory}{g}/{f}", f"{directory}{g}")

In [ ]:
train_directory = directory
train_data_generator = ImageDataGenerator(rescale=1./255)
train_generator = train_data_generator.flow_from_directory(train_directory, target_size=(
    288, 432), color_mode="rgba", class_mode='categorical', batch_size=128)

validation_directory = f"{directory[:-6]}test/"
validation_data_generator = ImageDataGenerator(rescale=1./255)
validation_generator = validation_data_generator.flow_from_directory(validation_directory, target_size=(
    288, 432), color_mode='rgba', class_mode='categorical', batch_size=128)

In [ ]:
import keras.backend as K

def cnn(input_shape=(288, 432, 4), classes=9):
    def step(dim, X):
        X = Conv2D(dim, kernel_size=(3, 3), strides=(1, 1))(X)
        X = BatchNormalization(axis=3)(X)
        X = Activation('relu')(X)
        return MaxPooling2D((2, 2))(X)
    X_input = Input(input_shape)
    X = X_input
    layer_dims = [8, 16, 32, 64, 128, 256]
    for dim in layer_dims:
        X = step(dim, X)

    X = Flatten()(X)
    X = Dense(classes, activation='softmax',
              name=f'fc{classes}',  kernel_initializer=glorot_uniform(seed=9))(X)
    model = Model(inputs=X_input, outputs=X, name='cnn')
    return model

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [ ]:
model = cnn(input_shape=(288, 432, 4), classes=9)
opt = Adam(learning_rate=0.00005)
model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=['accuracy', f1_score])
model.summary()

In [ ]:
model.fit(train_generator, epochs=40, validation_data=validation_generator)

In [ ]:
model.save('CNNModelWeights', save_format='h5')